In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, greatest, coalesce, lit, first, when
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import socket
import time

# --- Configuração do Spark Session ---
local_ip = socket.gethostbyname(socket.gethostname())

spark = SparkSession.builder \
    .appName("Workload 1 - Correlacao por Genero e Ano") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("Spark Session iniciada com sucesso!")
print("Conectado ao Spark master:", spark.sparkContext.master)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 04:20:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session iniciada com sucesso!
Conectado ao Spark master: spark://spark-master:7077


In [2]:
#Definição e Execução do Workload

def executar_workload1():
    """
    Executa o Workload 1:
    1. Lê o dataset limpo do Parquet.
    2. Calcula a correlação entre a Popularidade e outras métricas numéricas,
       agrupado por gênero e ano.
    3. Identifica a métrica com a maior correlação para cada grupo.
    4. Salva o resultado em um único arquivo CSV.
    """
    
    # --- 1. Carregar os Dados ---
    caminho_parquet = "/spark-data/musicas_limpas.parquet"
    df = spark.read.parquet(caminho_parquet)

    # Colunas numéricas para análise de correlação
    numeric_cols = [
        "Tempo", "Loudness (db)", "Energy", "Danceability", "Positiveness",
        "Speechiness", "Liveness", "Acousticness", "Instrumentalness"
    ]

    # --- 2. Calcular Correlações de forma Otimizada ---
    # Cria uma expressão de correlação para cada coluna numérica com a Popularidade
    corr_exprs = [
        expr(f"corr(Popularity, `{c}`)").alias(f"corr_{c}") for c in numeric_cols
    ]

    # Agrupa por gênero e ano e calcula todas as correlações de uma vez
    df_correlacoes = df.groupBy("main_genre", "release_year").agg(*corr_exprs)

    # --- 3. Encontrar a Métrica de Maior Correlação ---
    # Nomes das novas colunas de correlação (ex: "corr_Energy")
    corr_col_names = [f"corr_{c}" for c in numeric_cols]

    # Encontra o valor máximo de correlação na linha
    max_corr_value = greatest(*[col(c) for c in corr_col_names])

    # Encontra o nome da coluna original que corresponde ao valor máximo
    # A função coalesce retorna a primeira coluna que não é nula
    max_corr_col_name = coalesce(*[
        when(col(c) == max_corr_value, lit(c.replace("corr_", "")))
        for c in corr_col_names
    ])

    df_resultado = df_correlacoes.withColumn("maior_corr_coluna", max_corr_col_name)

    # --- 4. Preparar e Salvar o Resultado Final ---
    df_final = df_resultado.select("main_genre", "release_year", "maior_corr_coluna")
    
    print(f"Workload 1 concluído.")
    df_final.show(10)

In [3]:
# --- Medição do Tempo de Execução ---
print("Iniciando a execução do Workload 1...")
inicio = time.time()

executar_workload1()

fim = time.time()
duracao = fim - inicio

print(f"\n--- Análise de Desempenho ---")
print(f"Tempo total de execução: {duracao:.2f} segundos")
print(f"----------------------------")

Iniciando a execução do Workload 1...


Workload 1 concluído.


25/07/09 04:20:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+----------------+------------+-----------------+
|      main_genre|release_year|maior_corr_coluna|
+----------------+------------+-----------------+
|            soul|        1981|     Acousticness|
|           metal|        2008|     Danceability|
|             emo|        2016|           Energy|
|             rnb|        1974|           Energy|
|           house|        2019| Instrumentalness|
|       indie pop|        1998|     Acousticness|
|       deathcore|        2019|           Energy|
|           swing|        2002|            Tempo|
|             pop|        1950|     Acousticness|
|progressive rock|        2001|     Danceability|
+----------------+------------+-----------------+
only showing top 10 rows


--- Análise de Desempenho ---
Tempo total de execução: 32.72 segundos
----------------------------


In [4]:
spark.stop()